In [13]:
from ultralytics import YOLO
import cv2
import numpy as np
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import mediapipe as mp
import os

In [ ]:
def trim_and_select_enter_frames(video_path, num_output_frames=20, end_padding=20, fixed_start_frame=30, yolo_model=None):
    if yolo_model is None:
        yolo_model = YOLO("yolov8m ska visa.pt")
    
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    detections = []
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = yolo_model.track(frame, persist=True, classes=[0], tracker="bytetrack.yaml", verbose=False)
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            ids = results[0].boxes.id.cpu().numpy()
            for box, track_id in zip(boxes, ids):
                detections.append({
                    "frame": frame_idx,
                    "id": int(track_id),
                })
        
        frame_idx += 1
    cap.release()

    if not detections:
        print("No detections found in the video.")
        return []
    
    id_counts = Counter(d['id'] for d in detections)
    main_id = id_counts.most_common(1)[0][0]
    track = [d for d in detections if d['id'] == main_id]

    start_frame = fixed_start_frame    
    last_frame = track[-1]["frame"]
    end_frame = min(last_frame + end_padding, total_frames - 1)

    if end_frame <= start_frame:
        print("trimmed range is invalid.")
        return []
    
    selected_frames = np.linspace(start_frame, end_frame, num_output_frames, dtype=int).tolist()

    return selected_frames

In [26]:
def trim_and_select_pass_frames(
        video_path, 
        num_output_frames=20, 
        end_padding=40, 
        fixed_start_frame=30,
        yolo_model=None,
        stop_window=30,
        horizontal_thresh=5):
    
    if yolo_model is None:
        yolo_model = YOLO("yolov8l.pt")

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_idx = 0
    track_history = defaultdict(list)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = yolo_model.track(frame, persist=True, classes=[0], tracker="bytetrack.yaml", verbose=False)
        
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            ids = results[0].boxes.id.cpu().numpy()
            for box, track_id in zip(boxes, ids):
                x1, y1, x2, y2 = box
                center_x = (x1 + x2) / 2
                track_history[int(track_id)].append((frame_idx, center_x))
        frame_idx += 1
    
    cap.release()

    if not track_history:
        print("No significant motion detected in the video.")
        return []
    
    main_id = max(track_history, key=lambda k: len(track_history[k]))
    main_track = track_history[main_id]

    main_track.sort()

    for i in range(len(main_track) -stop_window -1, 0, -1):
        recent_positions = [x for _, x in main_track[i:i+stop_window]]
        max_disp = max(recent_positions) - min(recent_positions)
        if max_disp > horizontal_thresh:
            stop_frame = main_track[i + stop_window][0]
            break
    else:
        stop_frame = main_track[-1][0]
    
    
    start_frame = fixed_start_frame
    end_frame = min(stop_frame - end_padding, total_frames - 1)

    if end_frame <= start_frame:
        print("trimmed range is invalid.")
        return []
    
    selected_frames = np.linspace(start_frame, end_frame, num_output_frames, dtype=int).tolist()
    return selected_frames

In [39]:
def save_trimmed_video(video_path, selected_frames, output_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    selected_set = set(selected_frames)
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx in selected_set:
            out.write(frame)
        frame_idx += 1
    
    cap.release()
    out.release()
    print(f"Trimmed video saved to {output_path}")

In [40]:
def process_videos(input_dir, output_dir, yolo_small_model, yolo_large_model):
    os.makedirs(output_dir, exist_ok=True)

    for folder_name in os.listdir(input_dir):
        folder_path = os.patj.join(input_dir, folder_name)
        if not os.path.isdir(folder_path):
            continue

        for filename in os.listdir(folder_path):
            if not filename.lower().endswith(('.mp4', '.avi', '.mov')):
                continue

            video_path = os.path .join(folder_path, filename)

            if "enter" in folder_name.lower():
                selected_frames = trim_and_select_enter_frames(
                    video_path,
                    num_output_frames=20,
                    end_padding=20,
                    fixed_start_frame=30,
                    yolo_model=yolo_small_model)
                subfolder = "processed_enter"

            elif "pass" in folder_name.lower():
                selected_frames = trim_and_select_pass_frames(
                    video_path,
                    num_output_frames=20,
                    end_padding=40,
                    fixed_start_frame=30,
                    yolo_model=yolo_large_model,
                    stop_window=30,
                    horizontal_thresh=5)
                subfolder = "processed_pass"
            else:
                print(f"Skipping {filename}, unknown label")
                continue

            if not selected_frames:
                print(f"No valid frames selected for {filename}.")
                continue

            output_folder = os.path.join(output_dir, subfolder)
            os.makedirs(output_folder, exist_ok=True)

            base_name, ext = os.path.splitext(filename)
            output_path = os.path.join(output_folder, f"{base_name}_trimmed{ext}")

            save_trimmed_video(video_path, selected_frames, output_path)

In [37]:
selected_frames = trim_and_select_pass_frames("C:/Users/hanna/Documents/Thesis/datainsamling/data/SLOW_front_pass/front_pass_2.mp4")

In [9]:
def show_frame(video_path, frame_number):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    if not ret:
        print(f"Could not read frame {frame_number}")
        return

    cv2.imshow(f"Frame {frame_number}", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cap.release()

In [38]:
for frame in selected_frames:
    show_frame("C:/Users/hanna/Documents/Thesis/datainsamling/data/SLOW_front_pass/front_pass_2.mp4", frame)

In [3]:
def extract_and_save_frame(video_path, selected_frames, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    selected_set = set(selected_frames)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx in selected_set:
            filename = os.path.join(save_dir, f"frame_{frame_idx:04d}.jpg")
            cv2.imwrite(filename, frame)

        frame_idx += 1
    cap.release()